In [87]:
import os
#from dotenv import load_dotenv
import requests
import xmltodict, json
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import pandas as pd
import seaborn as sns
#from wordcloud import WordCloud
#from textwrap import wrap

**<font color='teal'>get api key from .env file</font>**

In [1]:
load_dotenv()
API_KEY = os.getenv('api.env')

print(API_KEY)

ModuleNotFoundError: No module named 'dotenv'

**<font color='teal'>Use Global Giving's API to access all active projects with respective themes</font>**

In [36]:
# Assign URL to variable: url
url = 'https://api.globalgiving.org/api/public/projectservice/themes/projects/ids?api_key=efee163e-82d8-413b-955e-c2dcc7b4bfba'

# Package the request, send the request and catch the response: r
r = requests.get(url)

**<font color='teal'>The response is in xml format, so we will use xmltodict to convert it to a dictionary</font>**

In [37]:
data_dict = xmltodict.parse(r.text)

**<font color='teal'>Next I will take a look at the keys find what will lead me to the themes and the project ID's</font>**

In [38]:
print(data_dict['themes']['theme'][27]['name'])
print(data_dict['themes']['theme'][27]['projects']['project'][430]['id'])

Wildlife Conservation
20480


**<font color='teal'>Next I will use a for loop to get a dictionary with the just the themes and their respective projects. This dictionary will be named "theme_and_proj_id"</font>**

In [39]:
theme_and_proj_id = {}

for i in range(len(data_dict['themes']['theme'])):
    theme = data_dict['themes']['theme'][i]['name']
    project_id_list = []
    for j in range(len(data_dict['themes']['theme'][i]['projects']['project'])):
        project_id_list.append(int(data_dict['themes']['theme'][i]['projects']['project'][j]['id']))
    theme_and_proj_id[theme] = project_id_list

theme_and_proj_id.keys()

dict_keys(['Animal Welfare', 'Child Protection', 'Climate Action', 'Peace and Reconciliation', 'Disaster Response', 'Economic Growth', 'Education', 'Ecosystem Restoration', 'Gender Equality', 'Physical Health', 'Ending Human Trafficking', 'Justice and Human Rights', 'Sport', 'Digital Literacy', 'Food Security', 'Arts and Culture', 'LGBTQIA+ Equality', 'COVID-19', 'Clean Water', 'Disability Rights', 'Ending Abuse', 'Mental Health', 'Racial Justice', 'Refugee Rights', 'Reproductive Health', 'Safe Housing', 'Sustainable Agriculture', 'Wildlife Conservation'])

**<font color='teal'>Next I want to access the summary, challenge, solution, and long term impact of a [project](https://www.globalgiving.org/projects/trees-for-tribes-1/) using Global Giving's API.</font>**

In [40]:
# Assign URL to variable: url
url = 'https://api.globalgiving.org/api/public/projectservice/projects/23245?api_key=efee163e-82d8-413b-955e-c2dcc7b4bfba'

# Package the request, send the request and catch the response: r
project = requests.get(url)

# Print the text of the response
projectinfo_dict = xmltodict.parse(project.text)
theme = projectinfo_dict['project']['themeName']
longtermimpact = projectinfo_dict['project']['longTermImpact']
challenge = projectinfo_dict['project']['need']
summary = projectinfo_dict['project']['summary']
solution = projectinfo_dict['project']['activities']
title = projectinfo_dict['project']['title']

In [41]:
theme

'Food Security'

In [42]:
title

'Bring Fruit Trees to Native American Reservations'

In [43]:
summary

"Trees for Tribes supports Native American communities' efforts toward food sovereignty & food justice by planting fruit tree orchards that provide abundant harvests of nutritious fruit. Tribal members also build their skills, (re)learning important techniques on how to care for their orchards & preserve the harvests. Through this program, FTPF has planted dozens of orchards filled with hundreds of fruit trees & provided arboricultural training for tribal communities all across Turtle Island."

In [44]:
challenge

'Hunger is a growing problem for Native American families. 23% of American Indian households experience food insecurity. The very youngest tribal members are most greatly affected by hunger & poverty, with 43% of children under the age of 5 living in poverty. With poverty and food insecurity also come obesity and related challenges, including heart disease and diabetes. This project will most benefit tribal communities of all ages, supporting healthy lifelong choices and access to fresh foods.'

In [45]:
solution

'Encouraging consumption of fresh foods by making it easier to access those foods is one key to improving food security and health for very low-income Native Americans. Trees for Tribes addresses these issues by providing fruit tree orchards to Native American communities. Volunteerism is encouraged by offering household mini-orchards to each family that helps plant the community orchards. FTPF also provides workshops on tree care so that participants can care for their new trees.'

In [46]:
longTermImpact

'More than 1,000 Native American families will have ready access to fresh organic produce as they harvest the fruit from their orchards. They will also be reintroduced the skills of self-reliance long embedded in Indigenous food systems as they integrate the orchards into tribal agricultural systems. Young people will begin to learn about how fresh foods are the most healthful choices and have easy access to fruit during their school day.'

**<font color='teal'>Here I will combine the title, summary, challenge, and solution into one long text, which I will clean and get ready for NLP</font>**

In [66]:
text = title + ' ' + summary + ' ' + challenge + ' ' + solution + ' ' + longTermImpact

#lowercase the text
text=text.lower()

#remove numbers
text = ''.join([i for i in text if not i.isdigit()])

#remove punctuation
punct = string.punctuation
text = text.translate(str.maketrans('','', punct))

#remove extra spaces
text = " ".join(text.split())
text

'bring fruit trees to native american reservations trees for tribes supports native american communities efforts toward food sovereignty food justice by planting fruit tree orchards that provide abundant harvests of nutritious fruit tribal members also build their skills relearning important techniques on how to care for their orchards preserve the harvests through this program ftpf has planted dozens of orchards filled with hundreds of fruit trees provided arboricultural training for tribal communities all across turtle island hunger is a growing problem for native american families of american indian households experience food insecurity the very youngest tribal members are most greatly affected by hunger poverty with of children under the age of living in poverty with poverty and food insecurity also come obesity and related challenges including heart disease and diabetes this project will most benefit tribal communities of all ages supporting healthy lifelong choices and access to 

**<font color='teal'>Next, I will combine the steps above into a function that takes in a porject URL and outputs clean text</font>**

In [70]:
def get_clean_text(url):
    # Package the request, send the request and catch the response: r
    project = requests.get(url)

    # Print the text of the response
    projectinfo_dict = xmltodict.parse(project.text)
    theme = projectinfo_dict['project']['themeName']
    longtermimpact = projectinfo_dict['project']['longTermImpact']
    challenge = projectinfo_dict['project']['need']
    summary = projectinfo_dict['project']['summary']
    solution = projectinfo_dict['project']['activities']
    title = projectinfo_dict['project']['title']
    
    text = title + ' ' + summary + ' ' + challenge + ' ' + solution + ' ' + longTermImpact

    #lowercase the text
    text=text.lower()

    #remove numbers
    text = ''.join([i for i in text if not i.isdigit()])

    #remove punctuation
    punct = string.punctuation
    text = text.translate(str.maketrans('','', punct))

    #remove extra spaces
    text = " ".join(text.split())
    
    return text

In [72]:
popular_themes=['COVID-19','Racial Justice','Climate Action','Education','Gender Equality']

project_ids = []
themes = []
texts = []

for theme in popular_themes:
    IDs=random.sample(theme_and_proj_id[theme],10)
    for ID in IDs:
        themes.append(theme)
        project_ids.append(ID)
        url = 'https://api.globalgiving.org/api/public/projectservice/projects/{}?api_key=efee163e-82d8-413b-955e-c2dcc7b4bfba'.format(str(ID))
        text = get_clean_text(url)
        texts.append(text)
        
print(len(project_ids),len(themes),len(texts))

50 50 50


In [84]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(texts)
bw = pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names(),index=project_ids)
bw

,aba,ababa,abandoned,abhiyan,ability,able,about,above,abriendo,absolutely,...,yeah,year,years,yet,york,you,young,your,youth,youthled
49278,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
46904,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
55295,0,0,0,0,0,0,1,0,0,0,...,0,1,1,1,0,0,1,0,0,0
50443,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,4,0,3,0
48093,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
46503,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
46600,0,0,0,0,1,1,1,0,0,0,...,0,3,0,0,0,0,1,0,0,0
48000,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
48735,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,1,0,0,0
46857,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


KeyError: 'count'

In [85]:
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(texts)
tfid = pd.DataFrame(trsfm.toarray(),columns=vectorizer.get_feature_names(),index=project_ids)
tfid

,aba,ababa,abandoned,abhiyan,ability,able,about,above,abriendo,absolutely,...,yeah,year,years,yet,york,you,young,your,youth,youthled
49278,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019913,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019913,0.000000,0.000000,0.00000
46904,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015613,0.066177,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015613,0.000000,0.000000,0.00000
55295,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014888,0.000000,0.000000,0.000000,...,0.000000,0.039145,0.042467,0.046750,0.000000,0.000000,0.014888,0.000000,0.000000,0.00000
50443,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019282,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.077126,0.000000,0.172719,0.00000
48093,0.000000,0.000000,0.000000,0.000000,0.000000,0.052961,0.016866,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.052961,0.000000,0.000000,0.016866,0.000000,0.000000,0.00000
46503,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019593,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019593,0.000000,0.000000,0.00000
46600,0.000000,0.000000,0.000000,0.000000,0.044217,0.044217,0.014081,0.000000,0.000000,0.000000,...,0.000000,0.111071,0.000000,0.000000,0.000000,0.000000,0.014081,0.000000,0.000000,0.00000
48000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019536,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019536,0.058333,0.000000,0.00000
48735,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027817,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013909,0.000000,0.000000,0.00000
46857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019714,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019714,0.000000,0.000000,0.00000
